In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

Convertendo informações categoricas (sexo, fumante, região) para 1 ou 0.

In [ ]:
dataset_encoded = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'])

Separando todas as informações sobre as pessoas, exceto a informação sobre as despesas médicas.

Equivalente as questões das prova e as respostas, para a nossa maquina.

In [ ]:
X = dataset_encoded.drop('expenses', axis=1)
y = dataset_encoded['expenses']

Dividindo 80% dos dados para treinamento (X_train, y_train), e 20% para teste (X_test, test_labels).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

 Padronizando as características, removendo a média e dimensionando para a variância unitária. É aplicado separadamente aos conjuntos de treinamento e teste para evitar vazamento de dados.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
test_dataset = scaler.transform(X_test)

Definindo um modelo de rede neural feedforward usando o Keras. O modelo possui várias camadas densas, cada uma com um número específico de neurônios e função de ativação.

E compila o modelo para treinamento.

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='relu')
])

model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])

O modelo é treinado usando os dados de treinamento.

Avalia o modelo no conjunto de dados de teste para obter métricas como Erro Médio Absoluto (MAE) e Erro Médio Quadrático (MSE).

E por fim, ele imprime a diferença média absoluta entre as despesas previstas e reais.

In [ ]:

history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, verbose=0, callbacks=[tfdocs.modeling.EpochDots()])

loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print("Testing set Mean Abs Error:", mae, "expenses")


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
